## 형태소 분석

### 1. Mecab 사용

In [ ]:
from konlpy.tag import Mecab

In [35]:
out = tokenizer.morphs('아버지가방에들어가신다')

TypeError: morphs() missing 1 required positional argument: 'phrase'

> 윈도우 프로 미만 버전에서는 지원 안 되는 것으로 생각됨

### 1_2. MeCab 사용

In [1]:
import MeCab

In [2]:
tokenizer = MeCab.Tagger()

In [3]:
out = tokenizer.parse('아버지가방에들어가신다\n오늘은 신나는 화요일.') #한 줄씩 넣기

In [140]:
out

'아버지\tNNG,*,F,아버지,*,*,*,*\n가\tJKS,*,F,가,*,*,*,*\n방\tNNG,장소,T,방,*,*,*,*\n에\tJKB,*,F,에,*,*,*,*\n들어가\tVV,*,F,들어가,*,*,*,*\n신다\tEP+EC,*,F,신다,Inflect,EP,EC,시/EP/*+ㄴ다/EC/*\n오늘\tNNG,*,T,오늘,*,*,*,*\n은\tJX,*,T,은,*,*,*,*\n신나\tVV,*,F,신나,*,*,*,*\n는\tETM,*,T,는,*,*,*,*\n화요일\tNNG,*,T,화요일,Compound,*,*,화/NNG/*+요일/NNG/*\n.\tSF,*,*,*,*,*,*,*\nEOS\n'

In [144]:
out = out.replace(',','')
con = re.sub('(\s.+\n)',' ',out) # 정규표현식으로 변환해서 형태소만 추출
con

'아버지 가 방 에 들어가 신다 오늘 은 신나 는 화요일 . EOS\n'

## 전체 데이터 형태소 분석

In [4]:
import re
from gensim.utils import to_unicode
import json
import pandas as pd
from bs4 import BeautifulSoup

In [5]:
# 불특정 문자 제거 함수 정의
WIKI_REMOVE_CHARS = re.compile("'+|(=+.{2,30}=+)|__TOC__|(ファイル:).+|:(en|de|it|fr|es|kr|zh|no|fi):|\n", re.UNICODE)
WIKI_SPACE_CHARS = re.compile("(\\s|゙|゚|　)+", re.UNICODE)
EMAIL_PATTERN = re.compile("(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)", re.UNICODE)
URL_PATTERN = re.compile("(ftp|http|https)?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", re.UNICODE)
WIKI_REMOVE_TOKEN_CHARS = re.compile("(\\*$|:$|^파일:.+|^;)", re.UNICODE)
MULTIPLE_SPACES = re.compile(' +', re.UNICODE)

In [6]:
# 전체 training data 취합

tokenizer = MeCab.Tagger()

output_fname = "../Data/output/processed_korquad_train_final.txt"
output_file = open(output_fname, 'w', encoding ='utf-8')

for i in range(39):
    corpus_fname = "../Data/input/korquad2.1_train_" + str(i) + ".json"
    
    with open(corpus_fname) as f1:
        dataset_json = json.load(f1)
        dataset = dataset_json['data']
        
        for article in dataset:
            w_lines = []
            paragraph = article['context']
            
            # 불필요한 단어,문자 제거
            paragraph = re.sub('(\n+)','\n', paragraph) # \n이 두번이상 나오는 부분 한번으로 줄임
            paragraph = re.sub(EMAIL_PATTERN, ' ', paragraph)  # remove email pattern
            paragraph = re.sub(URL_PATTERN, ' ', paragraph) # remove url pattern
            #paragraph = re.sub(WIKI_REMOVE_CHARS, ' ', paragraph)  # remove unnecessary chars
            paragraph = re.sub(WIKI_SPACE_CHARS, ' ', paragraph) 
            paragraph = re.sub(MULTIPLE_SPACES, ' ', paragraph)
            
            # 형태소 분석 (이 과정에서 중간의 \n 들은 다 제거됨)
            paragraph = tokenizer.parse(paragraph)
            paragraph = paragraph.replace(',','')
            paragraph = re.sub('(\s.+\n)',' ', paragraph) # 정규표현식으로 변환해서 형태소만 추출
            w_lines.append(paragraph)
            
            for qa in article['qas']:
                q_text = qa['question']
                a_text = qa['answer']['text']
                qas = q_text + " " + a_text
                
                # 형태소 분석
                qas = tokenizer.parse(qas)
                qas = qas.replace(',','')
                qas = re.sub('(\s.+\n)',' ', qas) # 정규표현식으로 변환해서 형태소만 추출
                w_lines.append(qas)
                
            for line in w_lines:
                output_file.writelines(line)
        

In [6]:
# 전체 dev data 취합
output_fname = "../Data/output/processed_korquad_dev_all.txt"
output_file = open(output_fname, 'w', encoding ='utf-8')

for i in range(5):
    corpus_fname = "../Data/input/korquad2.1_dev_" + str(i) + ".json"
    
    with open(corpus_fname) as f1:
        dataset_json = json.load(f1)
        dataset = dataset_json['data']
        
        for article in dataset:
            w_lines = []
            paragraph = article['context']
            
            # 불필요한 단어,문자 제거
            paragraph = re.sub('(\n+)','\n', paragraph) # \n이 두번이상 나오는 부분 한번으로 줄임
            paragraph = re.sub(EMAIL_PATTERN, ' ', paragraph)  # remove email pattern
            paragraph = re.sub(URL_PATTERN, ' ', paragraph) # remove url pattern
            #paragraph = re.sub(WIKI_REMOVE_CHARS, ' ', paragraph)  # remove unnecessary chars
            paragraph = re.sub(WIKI_SPACE_CHARS, ' ', paragraph) 
            paragraph = re.sub(MULTIPLE_SPACES, ' ', paragraph)
            
            # 형태소 분석 (이 과정에서 중간의 \n 들은 다 제거됨)
            paragraph = tokenizer.parse(paragraph)
            paragraph = paragraph.replace(',','')
            paragraph = re.sub('(\s.+\n)',' ', paragraph) # 정규표현식으로 변환해서 형태소만 추출
            w_lines.append(paragraph)
            
            for qa in article['qas']:
                q_text = qa['question']
                a_text = qa['answer']['text']
                qas = q_text + " " + a_text
                
                # 형태소 분석
                qas = tokenizer.parse(qas)
                qas = qas.replace(',','')
                qas = re.sub('(\s.+\n)',' ', qas) # 정규표현식으로 변환해서 형태소만 추출
                w_lines.append(qas)
                
            for line in w_lines:
                output_file.writelines(line)
        

In [16]:
output_fname = open("../Data/output/processed_korquad_train_final.txt",'r', encoding ='utf-8')
    
output_fname.readline()

'<! DOCTYPE html > < html > < head > < meta > < title > 예 고범 - 위키백과  우리 모두 의 백과사전 </ title > < link > < meta > < link > < meta > < meta > < meta > < meta > < link > < link > < link > < link > < link > < link > < link > < link > < link > < link > < link > </ head > < body > < div ></ div > < div ></ div > < div > < a ></ a > < div ></ div > < div > </ div > < h 1 > 예 고범 </ h 1 > < div > < div > 위키백과  우리 모두 의 백과사전 . </ div > < div ></ div > < div ></ div > < a > 둘러보 기 로 가 기 </ a > < a > 검색 하 러 가 기 </ a > < div >< div >< p >《< b > 예 고범 </ b >》(< span >< a > 일본어 </ a >: </ span >< span > 予 告 犯 </ span >) 은  츠츠이 테츠야 의 < a > 만화 </ a > 작품 이 다 . </ p > < div >< input >< div >< h 2 > 목차 </ h 2 >< span >< label ></ label ></ span ></ div > < ul > < li >< a >< span > 1 </ span > < span > 개요 </ span ></ a ></ li > < li >< a >< span > 2 </ span > < span > 등장인물 </ span ></ a > < ul > < li >< a >< span > 2 . 1 </ span > < span > 예 고범 그룹 </ span ></ a ></ li > < li >< a >< span > 2 . 2 </ span > < spa